In [1]:
import numpy as np
import pandas as pd
import random

In [47]:
# https://www.kaggle.com/dylanjcastillo/7k-books-with-metadata
df = pd.read_csv("data/dataset.csv")

df = df.drop('thumbnail',axis=1)
df = df.drop('isbn10',axis=1)
df = df.rename(columns={'isbn13': 'isbn'})
df.index = np.arange(1, len(df) + 1)

df['published_year'] = df['published_year'].astype(dtype=pd.Int32Dtype(), errors='ignore')
df['num_pages'] = df['num_pages'].astype(dtype=pd.Int32Dtype(), errors='ignore')
df['ratings_count'] = df['ratings_count'].astype(dtype=pd.Int32Dtype(), errors='ignore')

df['book_thickness'] = [ random.sample([0.12,0.105,0.11,0.09,0.08,0.065,0.06],1)[0] for i in range(len(df))] * df['num_pages'] + 2 * np.random.randint(0, 6, len(df))

df['book_thickness'] = df['book_thickness'].round(0).astype(dtype=pd.Int32Dtype(), errors='ignore')


df['title'] = np.where(df['subtitle'].notnull(), (df['title'] + ". " + df['subtitle']),df['title'])
df = df.drop('subtitle',axis=1)
df = df.dropna(how='any',axis=0)

df['price'] = df['book_thickness'] * 0.45 - df['ratings_count'] * 0.00001
df['price'] = df['price'].apply(lambda x: 2 if x < 2 else x)
df['price'] = df['price'].round(2)

df['num_pages'] = df['num_pages'].apply(lambda x: 100 if x == 0 else x)
df['book_thickness'] = df['book_thickness'].apply(lambda x: 13 if x == 0 else x)

df['categories'] = df['categories'].str.replace('"', '')
data = dict()

for index, row in df.iterrows():
    if(str(row['categories']).lower().strip(' .;') in data):
        df.loc[index, 'categories'] = data[str(row['categories']).lower().strip(' .;')]
    elif(str(row['categories']).lower().strip(' .;') + 's' in data):
        df.loc[index, 'categories'] = data[str(row['categories']).lower().strip(' .;')+ 's']
    elif(str(row['categories']).lower().strip(' .;')[-1:] == 's' and str(row['categories']).lower().strip(' .;')[:-1] in data):
        df.loc[index, 'categories'] = data[str(row['categories']).lower().strip(' .;')[:-1]]
    else:
        data[str(row['categories']).lower().strip(' .;')] = row['categories']
        
df.head()

,isbn,title,authors,categories,description,published_year,average_rating,num_pages,ratings_count,book_thickness,price
1,9780002005883,Gilead,Marilynne Robinson,Fiction,A NOVEL THAT READERS and critics have been eag...,2004,3.85,247,361,24,10.80
2,9780002261982,Spider's Web. A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,2000,3.83,241,5164,37,16.60
3,9780006163831,The One Tree,Stephen R. Donaldson,American fiction,Volume Two of Stephen Donaldson's acclaimed se...,1982,3.97,479,172,35,15.75
4,9780006178736,Rage of angels,Sidney Sheldon,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",1993,3.93,512,29532,43,19.05
5,9780006280897,The Four Loves,Clive Staples Lewis,Christian life,Lewis' work on the nature of love divides love...,2002,4.15,170,33684,23,10.01


In [48]:
df_cat = pd.DataFrame(pd.unique(df['categories']),columns=['categories'])
df_cat['len'] = df_cat['categories'].str.len()
df_cat =  df_cat.sort_values('len')
df_cat = df_cat.drop('len',axis=1)

df_cat['dom_categories']= ""
for index, row in df_cat.iterrows():
    temp = str(row['categories']).lower()
    for index2, row2 in df_cat.iterrows():
        temp2 = str(row2['categories']).lower()
        if(temp != temp2):
            if temp in  list(map(lambda x: x.strip(' .;,)('), temp2.split())) or temp[:-1] in list(map(lambda x: x.strip(' .;,)('), temp2.split())):
                df_cat.loc[index2, 'dom_categories'] = str(df_cat.loc[index2, 'dom_categories'])+'$$'+str(row['categories'])

df_cat_final = pd.DataFrame(columns=['category','dom_category'])
i=0
for index, row in df_cat.iterrows():
    if(row['dom_categories'] != ''):
        for cat in row['dom_categories'][2:].split('$$'):
            df_cat_final.loc[i] = {'category' : row['categories'], 'dom_category' : cat}
            i+=1
            
    else:
        df_cat_final.loc[i] = {'category' : row['categories'], 'dom_category' : ''}
        i+=1            

In [49]:
from loremipsum import get_sentences

df_categor_tab = pd.DataFrame(columns=['category'])
df_categor_tab['category'] = pd.unique(df_cat_final['category'].copy())
sentences_list = get_sentences(len(df_categor_tab))
sentences_list
df_categor_tab["description"] =  pd.Series(sentences_list)
df_categor_tab

,category,description
0,Law,B'nibh' b'et' b'a' b'a' b'a' b'a' b'aptent'.
1,Art,B'urna' b'mi' b'a' b'pretium' b'tellus' b'a' b...
2,War,B'orci' b'id' b'a' b'a' b'nam' b'arcu' b'eros'...
3,Sex,B'diam' b'ut' b'a' b'eros' b'a' b'a' b'a' b'ni...
4,Men,B'urna' b'ac' b'a' b'arcu' b'sed' b'nec' b've'.
...,...,...
513,"Battle, Superintendent (Fictitious character)",B'urna' b'ut' b'eu' b'in' b'ac' b'id' b'a'.
514,"Detective and mystery comic books, strips, etc",B'ante' b'id' b'a' b'eros' b'a' b'a' b'vestibu...
515,Englisch - Geschichte - Lyrik - Aufsatzsammlung,B'cras' b'eu'.
516,Characters and characteristics in motion pictures,B'elit' b'ut' b'a' b'aptent' b'a' b'a' b'leo' ...


In [50]:
df_categor_tab.to_csv('sqldump/categories.csv', index=False)

In [51]:
df_cat_headrh = df_cat_final.replace(r'^\s*$', np.nan, regex=True).dropna(how='any',axis=0)
df_cat_headrh

,category,dom_category
92,Gay men,Men
119,Civil law,Law
134,City girl,Girls
155,Bus travel,Travel
177,New England,England
...,...,...
535,Death (Fictitious character : Gaiman),Death
543,"Detective and mystery stories, English",English
545,"Detective and mystery stories, American",Americans
548,African Americans in radio broadcasting,Americans


In [52]:
df_cat_headrh.to_csv('sqldump/categories_hierarchy.csv', index=False)

In [53]:
df_a = df[['isbn','authors']]
df_a_full = pd.DataFrame(columns=['isbn','author'])
df_a_full['isbn'] = df_a_full['isbn'].astype(dtype=np.int64, errors='ignore')

i=0
for index, row in df_a.iterrows():   
    for author in row['authors'].split(";"):
        df_a_full.loc[i] = {'isbn' : row['isbn'], 'author' : author}
        i+=1

In [54]:
df_a_full.to_csv('sqldump/book_author.csv', index=False)
df_a_full

,isbn,author
0,9780002005883,Marilynne Robinson
1,9780002261982,Charles Osborne
2,9780002261982,Agatha Christie
3,9780006163831,Stephen R. Donaldson
4,9780006178736,Sidney Sheldon
...,...,...
7857,9788185300535,Sri Nisargadatta Maharaj
7858,9788185300535,Sudhakar S. Dikshit
7859,9789027712059,Georg Wilhelm Friedrich Hegel
7860,9789042003408,Helena Grice


In [55]:
df_main = df.copy()
df_main = df_main.drop(['authors'],axis=1)
df_main.head()

,isbn,title,categories,description,published_year,average_rating,num_pages,ratings_count,book_thickness,price
1,9780002005883,Gilead,Fiction,A NOVEL THAT READERS and critics have been eag...,2004,3.85,247,361,24,10.80
2,9780002261982,Spider's Web. A Novel,Detective and mystery stories,A new 'Christie for Christmas' -- a full-lengt...,2000,3.83,241,5164,37,16.60
3,9780006163831,The One Tree,American fiction,Volume Two of Stephen Donaldson's acclaimed se...,1982,3.97,479,172,35,15.75
4,9780006178736,Rage of angels,Fiction,"A memorable, mesmerizing heroine Jennifer -- b...",1993,3.93,512,29532,43,19.05
5,9780006280897,The Four Loves,Christian life,Lewis' work on the nature of love divides love...,2002,4.15,170,33684,23,10.01


In [56]:
df_main.to_csv('sqldump/books.csv', index=False)

In [57]:
bookcase_types = pd.DataFrame(
    {'type': ['type_1','type_2','type_3'],
     'width': [1200,1800,2400],
     'shelf_numbers': [8,7,6]
    })
bookcase_types

,type,width,shelf_numbers
0,type_1,1200,8
1,type_2,1800,7
2,type_3,2400,6


In [58]:
bookcase_types.to_csv('sqldump/bookcase_types.csv', index=False)

In [59]:
pd.set_option('display.max_rows', 50)
bookcases = pd.DataFrame({'hall' : [],'bookcase':[],'bookcase_type':[]})
bookcases['hall'] = bookcases['hall'].astype(dtype=pd.Int32Dtype(), errors='ignore')
bookcases['bookcase'] = bookcases['bookcase'].astype(dtype=pd.Int32Dtype(), errors='ignore')

for i in range(1,6):
    for j in range(1,13):
        bookcases = bookcases.append({'hall' : i,'bookcase':j,'bookcase_type':list(bookcase_types['type'].sample(n=1))[0]}, ignore_index=True)
bookcases

,hall,bookcase,bookcase_type
0,1,1,type_1
1,1,2,type_3
2,1,3,type_2
3,1,4,type_2
4,1,5,type_1
...,...,...,...
55,5,8,type_3
56,5,9,type_2
57,5,10,type_2
58,5,11,type_2


In [60]:
bookcases.to_csv('sqldump/bookcases.csv', index=False)


In [61]:
shelves = bookcases.copy()
shelves = shelves.drop(shelves.index[:])
shelves = shelves.drop('bookcase_type',axis=1)
for index, row in bookcases.iterrows():
    r_type = bookcase_types[bookcase_types['type'] == row['bookcase_type']]
    width = list(r_type['width'])[0]
    n = list(r_type['shelf_numbers'])[0]
    
    for i in range(1,n+1):
        shelves = shelves.append({'hall' : row['hall'],'bookcase':row['bookcase'],'shelf':i,'max_w':width}, ignore_index=True)

shelves = shelves.astype(int)
shelves

,hall,bookcase,max_w,shelf
0,1,1,1200,1
1,1,1,1200,2
2,1,1,1200,3
3,1,1,1200,4
4,1,1,1200,5
...,...,...,...,...
427,5,12,1800,3
428,5,12,1800,4
429,5,12,1800,5
430,5,12,1800,6


In [62]:
sum(shelves.max_w)

706800

In [63]:
books_shelf = df_main[['isbn','book_thickness']].copy()
books_shelf['numbers'] = np.random.randint(0, 6, len(books_shelf))
books_shelf['hall'] = -1
books_shelf['bookcase'] = -1
books_shelf['shelf'] = -1
books_shelf

,isbn,book_thickness,numbers,hall,bookcase,shelf
1,9780002005883,24,4,-1,-1,-1
2,9780002261982,37,4,-1,-1,-1
3,9780006163831,35,3,-1,-1,-1
4,9780006178736,43,3,-1,-1,-1
5,9780006280897,23,3,-1,-1,-1
...,...,...,...,...,...,...
6804,9788173031014,26,1,-1,-1,-1
6805,9788179921623,18,0,-1,-1,-1
6806,9788185300535,58,1,-1,-1,-1
6809,9789027712059,23,3,-1,-1,-1


In [64]:
(books_shelf['book_thickness'] * books_shelf['numbers']) .sum(),(books_shelf['book_thickness'] * books_shelf['numbers']) .max()

(577466, 1120)

In [65]:
i = 0
for index, row in books_shelf.iterrows():
    flag = True
    while flag and i < len(shelves):
        shelf = shelves.iloc[i]
        if (shelf['max_w'] - row['numbers'] * row['book_thickness'] >= 0) : 
            shelves.loc[i, 'max_w'] = shelf['max_w'] - row['numbers'] * row['book_thickness']
            books_shelf.loc[index, 'hall'] = shelf['hall']
            books_shelf.loc[index, 'bookcase'] = shelf['bookcase']
            books_shelf.loc[index, 'shelf'] = shelf['shelf']
            flag = False
        else:
            i = i+1
#         flag = False

books_shelf = books_shelf.drop('book_thickness',axis=1)
books_shelf = books_shelf.drop(books_shelf.loc[(books_shelf[['numbers']]==0).any(axis=1)].index)
books_shelf = books_shelf.drop(books_shelf.loc[(books_shelf[['shelf']]==-1).any(axis=1)].index)
books_shelf = books_shelf.reindex(columns=['isbn', 'hall', 'bookcase', 'shelf','numbers'])
books_shelf

,isbn,hall,bookcase,shelf,numbers
1,9780002005883,1,1,1,4
2,9780002261982,1,1,1,4
3,9780006163831,1,1,1,3
4,9780006178736,1,1,1,3
5,9780006280897,1,1,1,3
...,...,...,...,...,...
6803,9788172235222,5,5,4,3
6804,9788173031014,5,5,4,1
6806,9788185300535,5,5,4,1
6809,9789027712059,5,5,4,3


In [66]:
pd.set_option('display.max_rows', None)
shelves

,hall,bookcase,max_w,shelf
0,1,1,212,1
1,1,1,24,2
2,1,1,90,3
3,1,1,25,4
4,1,1,3,5
5,1,1,22,6
6,1,1,88,7
7,1,1,103,8
8,1,2,68,1
9,1,2,162,2


In [67]:
pd.set_option('display.max_rows', 50)

In [68]:
books_shelf.to_csv('sqldump/books_shelf.csv', index=False)